In [45]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [46]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [47]:
# URL to find information on Coronavirus
url = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus"

In [48]:
# Visiting the url 
browser.visit(url)

In [49]:
time.sleep(5)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [50]:
# All the Texas counties
counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
rows = counties.find_all('span')




In [51]:
counter = 0

county = []
cases = []

for row in rows:
    try:
        print(rows[counter].text.strip().split('\n'))
        county.append(rows[counter].text.strip().split('\n')[0])
        cases.append(rows[counter].text.strip().split('\n')[1])
        counter = counter + 3
        
    except:
        print('failure')

['Dallas County', '169']
['Harris County', '134']
['Travis County', '98']
['Tarrant County', '71']
['Bexar County', '69']
['Fort Bend County', '46']
['Collin County', '45']
['Denton County', '30']
['McLennan County', '23']
['Montgomery County', '23']
['Galveston County', '21']
['Brazoria County', '19']
['Williamson County', '19']
['Bell County', '18']
['Brazos County', '16']
['Smith County', '14']
['Lubbock County', '12']
['El Paso County', '11']
['Nueces County', '10']
['Webb County', '8']
['Hays County', '7']
['Cameron County', '6']
['Ellis County', '6']
['Matagorda County', '6']
['Comal County', '5']
['Wichita County', '5']
['Guadalupe County', '4']
['Castro County', '3']
['Hockley County', '3']
['Jefferson County', '3']
['Johnson County', '3']
['Midland County', '3']
['Wharton County', '3']
['Atascosa County', '2']
['Brown County', '2']
['Deaf Smith County', '2']
['Eastland County', '2']
['Grayson County', '2']
['Grimes County', '2']
['Hardin County', '2']
['Hidalgo County', '2']
[

In [52]:
df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Dallas County,169
1,Harris County,134
2,Travis County,98
3,Tarrant County,71
4,Bexar County,69
...,...,...
77,Tom Green County,1
78,Upshur County,1
79,Van Zandt County,1
80,Walker County,1


In [53]:
# Droping the word 'County'
df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [54]:

# df_stats.to_csv('static/csv/stats.csv', index=False)

In [55]:
browser.visit(url3)

In [56]:
harris_table = browser.find_by_id('covidtable')

In [57]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [58]:
harris_soup

<html lang="en-US" style="height: 100%;"><head id="Head">
<!--*********************************************-->
<!-- DNN Platform - http://www.dnnsoftware.com   -->
<!-- Copyright (c) 2002-2018, by DNN Corporation -->
<!--*********************************************-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="F0PqTcRKcnKG7nlC0FL6ptMucbfv5jV5cOPHE7k50sw" name="google-site-verification"/><title>
	2019 Novel Coronavirus
</title><meta content=",DotNetNuke,DNN" id="MetaKeywords" name="KEYWORDS"/><meta content="DotNetNuke " id="MetaGenerator" name="GENERATOR"/><meta content="INDEX, FOLLOW" id="MetaRobots" name="ROBOTS"/><link href="/Resources/Shared/stylesheets/dnndefault/7.0.0/default.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Resources/Search/SearchSkinObjectPreview.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Portals/27/skins/hcph/skin.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Portals/27/Contai

In [59]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [60]:
columns = ['Sex', 'Location', 'Age_Range', 'Status']


In [61]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [62]:
harris_df['Sex'] = harris_df['Sex'].apply(lambda x : x.split()[2])

In [63]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [64]:
harris_df.head()

,Sex,Location,Age_Range,Status
0,Female,NW,40-49,Confirmed (community spread)
1,Female,NW,60-69,Confirmed (community spread)
2,Female,NW,20-29,Confirmed (community spread)
3,Female,NW,20-29,Confirmed (domestic travel)
4,Male,NW,40-49,Confirmed (community spread)


In [65]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [66]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [67]:
df_counties = list(df['Texas_County'].apply(lambda x : x.split()[0]))

In [68]:
Counties_with_Cases = []

for i in df_counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Dallas---found
Harris---found
Travis---found
Tarrant---found
Bexar---found
Houston we have a problem: Fort
Collin---found
Denton---found
McLennan---found
Montgomery---found
Galveston---found
Brazoria---found
Williamson---found
Bell---found
Brazos---found
Smith---found
Lubbock---found
Houston we have a problem: El
Nueces---found
Webb---found
Hays---found
Cameron---found
Ellis---found
Matagorda---found
Comal---found
Wichita---found
Guadalupe---found
Castro---found
Hockley---found
Jefferson---found
Johnson---found
Midland---found
Wharton---found
Atascosa---found
Brown---found
Houston we have a problem: Deaf
Eastland---found
Grayson---found
Grimes---found
Hardin---found
Hidalgo---found
Kaufman---found
Kendall---found
Milam---found
Potter---found
Austin---found
Bastrop---found
Blanco---found
Bowie---found
Burnet---found
Cass---found
Chambers---found
Coryell---found
Crane---found
Houston we have a problem: De
Erath---found
Falls---found
Fannin---found
Fayette---found
Gaines---found
Gregg---f

In [69]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [70]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [71]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
...,...,...,...,...,...,...,...,...
71,Rusk,-94.761696,32.108032,201,48401,2.447868,0.232436,"(-94.761696, 32.108032)"
72,Terry,-102.335157,33.173737,223,48445,1.896687,0.222978,"(-102.335157, 33.173737)"
73,Upshur,-94.941320,32.736283,230,48459,1.799823,0.147655,"(-94.94132, 32.736283)"
74,Walker,-95.572242,30.738995,236,48471,2.152041,0.195448,"(-95.572242, 30.738995)"


In [72]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,169,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,134,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,98,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,71,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Bexar,69,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [73]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Dallas,169,32.766537,-96.777819
1,Harris,134,29.859671,-95.397821
2,Travis,98,30.334233,-97.781947
3,Tarrant,71,32.771852,-97.291165
4,Bexar,69,29.448845,-98.519663
...,...,...,...,...
71,Rusk,1,32.108032,-94.761696
72,Terry,1,33.173737,-102.335157
73,Upshur,1,32.736283,-94.941320
74,Walker,1,30.738995,-95.572242


In [74]:
# # Adding lat and longs to the dataframe
# df['lat'] = lats
# df['long'] = longs

In [75]:
# df.head()

In [76]:
browser.quit()

In [77]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [78]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)